# Extract Figueroa 2009 methylation signiture methylation probes

In [5]:
library(gdata)

gdata: read.xls support for 'XLS' (Excel 97-2004) files ENABLED.

gdata: read.xls support for 'XLSX' (Excel 2007+) files ENABLED.

Attaching package: ‘gdata’

The following object is masked from ‘package:stats’:

    nobs

The following object is masked from ‘package:utils’:

    object.size

The following object is masked from ‘package:base’:

    startsWith



In [11]:
n <- 16
clusters <- list()
for (i in seq(n)) {
    clusters[[i]] <- read.xls('../data/mmc3.xls', sheet=i, skip=1, header=T, stringsAsFactor=F)
}

In [13]:
head(clusters[[16]])

HELP_ID,REFSEQ,GENE.SYMBOL,Status
MSPI0406S00055568,NM_016178,OAZ3,Hypermethylated
MSPI0406S00055568,NM_031420,MRPL9,Hypermethylated
MSPI0406S00062872,NM_015569,DNM3,Hypermethylated
MSPI0406S00069595,NM_006618,JARID1B,Hypermethylated
MSPI0406S00083922,NM_014322,OPN3,Hypermethylated
MSPI0406S00099987,NM_173853,KRTCAP3,Hypermethylated


In [20]:
cluster.probes <- list()
for (i in seq(n)) {
    probe.names <- unique(clusters[[i]]$GENE.SYMBOL)
    cluster.probes[[i]] <- probe.names[probe.names!='']
}

In [22]:
lapply(cluster.probes, length)

[[1]]
[1] 173

[[2]]
[1] 63

[[3]]
[1] 210

[[4]]
[1] 128

[[5]]
[1] 58

[[6]]
[1] 176

[[7]]
[1] 124

[[8]]
[1] 118

[[9]]
[1] 114

[[10]]
[1] 374

[[11]]
[1] 124

[[12]]
[1] 289

[[13]]
[1] 138

[[14]]
[1] 310

[[15]]
[1] 196

[[16]]
[1] 124

In [ ]:
# save probes as separate tsv files
for (i in seq(n)) {
    write.table(cluster.probes[[i]], file=paste0('cluster', i, '.tsv', sep='\t', quote=F, row.names=F, col.names=F)
}

## Subset the methylation array given the gene symbol match

In [ ]:
# load methylation data (KNN imputation)